In [1]:
import numpy as np
import math
from numba import cuda, types

In [2]:
# Leave the values in this cell alone
M = 128
N = 32

# Input vectors of MxN and NxM dimensions
a = np.arange(M*N).reshape(M,N).astype(np.int32)
b = np.arange(M*N).reshape(N,M).astype(np.int32)
c = np.zeros((M, M)).astype(np.int32)

d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.to_device(c)

# NxN threads per block, in 2 dimensions
block_size = (N,N)
# MxM/NxN blocks per grid, in 2 dimensions
grid_size = (int(M/N),int(M/N))

print(block_size)
print(grid_size)

(32, 32)
(4, 4)


In [3]:
@cuda.jit
def mm(a, b, c):
    col, row = cuda.grid(2)
    stride_col, stride_row = cuda.gridsize(2)

    # `a_cache` and `b_cache` are already correctly defined
    #a_cache = cuda.shared.array(block_size, types.int32)
    #b_cache = cuda.shared.array(block_size, types.int32)

    for data_row in range(row, a.shape[0], stride_row):
        for data_col in range(col, b.shape[1], stride_col):
            sum = 0
            for i in range(a.shape[1]):
                sum += a[data_row][i] * b[i][data_col]
                
            c[data_row][data_col] = sum

In [4]:
@cuda.jit
def mm_shared(A, B, C):
    """
    使用Shared Memory的矩阵乘法 C = A * B
    """
    # 在Shared Memory中定义向量
    # 向量可被整个Block的所有Thread共享
    # 必须声明向量大小和数据类型
    sA = cuda.shared.array(block_size, types.int32)
    sB = cuda.shared.array(block_size, types.int32)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    row = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
    col = cuda.threadIdx.y + cuda.blockDim.y * cuda.blockIdx.y
    
    if row >= C.shape[0] and col >= C.shape[1]:
        # 当(x, y)越界时退出
        return

    tmp = 0
    BLOCK_SIZE = block_size[0]
    # 以一个 BLOCK_SIZE x BLOCK_SIZE 为单位
    for m in range(math.ceil(A.shape[1] / BLOCK_SIZE)):
        sA[tx, ty] = A[row, ty + m * BLOCK_SIZE]
        sB[tx, ty] = B[tx + m * BLOCK_SIZE, col]
        # 线程同步，等待Block中所有Thread预加载结束
        # 该函数会等待所有Thread执行完之后才执行下一步
        cuda.syncthreads()
        # 此时已经将A和B的子矩阵拷贝到了sA和sB

        # 计算Shared Memory中的向量点积
        # 直接从Shard Memory中读取数据的延迟很低
        for n in range(BLOCK_SIZE):
            tmp += sA[tx, n] * sB[n, ty]

        # 线程同步，等待Block中所有Thread计算结束
        cuda.syncthreads()

    # 循环后得到每个BLOCK的点积之和
    C[row, col] = tmp

In [5]:
# There's no need to update this kernel launch
%timeit mm_shared[grid_size, block_size](d_a, d_b, d_c)

145 µs ± 436 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
# Do not modify the contents in this cell
from numpy import testing
solution = a@b
output = d_c.copy_to_host()
# This assertion will fail until you correctly update the kernel above.
testing.assert_array_equal(output, solution)

In [7]:
%timeit mm[grid_size, block_size](d_a, d_b, d_c)

146 µs ± 455 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
# Do not modify the contents in this cell
from numpy import testing
solution = a@b
output_1 = d_c.copy_to_host()
# This assertion will fail until you correctly update the kernel above.
testing.assert_array_equal(output_1, solution)

In [9]:
print(output_1)

[[  1333248   1333744   1334240 ...   1395248   1395744   1396240]
 [  3364864   3366384   3367904 ...   3554864   3556384   3557904]
 [  5396480   5399024   5401568 ...   5714480   5717024   5719568]
 ...
 [255285248 255413744 255542240 ... 271347248 271475744 271604240]
 [257316864 257446384 257575904 ... 273506864 273636384 273765904]
 [259348480 259479024 259609568 ... 275666480 275797024 275927568]]
